In [2]:
import subprocess
from glob import glob
import os
import monai
from monai.networks.blocks import Warp
import nibabel as nib
import reg_mri
import numpy as np
import torch
from utils import getDevice

In [ ]:
mris = sorted(glob(os.path.join("dataset3", "GIN", "MRI", "*_affine.nii.gz")))
for i in range(len(mris)):
    ith = i+1
    cmd = 'python3 reg_train.py -a -w 1 0 1 -lr 0.1 -t null -e 250 -noaug -d ginaffine -o null_ith_' + str(ith) + '_gin'
    cmd = cmd.split(' ')
    cmd.extend(['-ith', str(ith)])
    subprocess.call(cmd)

In [8]:
mris = sorted(glob(os.path.join("dataset3", "GIN", "MRI", "*_affine.nii.gz")))
labels = sorted(glob(os.path.join("dataset3", "GIN", "Labels", "*_affine_convert.nii.gz")))
ith_models = sorted(glob(os.path.join("models", "ith_null", "null_ith_*.pth")))
print(ith_models)
outfolder = 'ith_null'
warp_nearest = Warp("nearest", "zeros")
atlas_name = "dataset3/Atlas/P56_Atlas_128_norm_id.nii.gz"    
atlas_labels = "dataset3/Atlas/P56_Annotation_128_norm_id.nii.gz"
ants_warp = glob(os.path.join("output", "Feminad", "ANTS", "DeformableWarp", "*.nii.gz"))
affine = nib.load(ants_warp[0]).affine
header = nib.load(ants_warp[0]).header         
affine_atlas = nib.load(atlas_labels).affine
header_atlas = nib.load(atlas_labels).header 
device = getDevice()
for i, _ in enumerate(ith_models): 
    ith_model = ith_models[i].split('models/')[-1]
    ith = int(ith_model.split('null_ith_')[-1].split('_')[0])
    print(ith_model)
    print(ith)
    print(mris[ith-1])
    print('-'*10)
    outname = "output/GIN/" + outfolder + "/MRI_N4_Registration_Deformable/" + mris[ith-1].split('/')[-1].split('.')[0]
    pred_images, ddfs = reg_mri.main(ith_model, mris[ith-1], outname, False, True, "null", newmodel=False, sym=False)
    
    pred_image = pred_images[0]
    #predsym_image = pred_images[1]
    moving_image = ddfs[0].cpu()
    
    deformable_ddf = ddfs[1].cpu()
    deformable_ddf = deformable_ddf.cpu().numpy()
    deformable_ddf = np.transpose(deformable_ddf, (2, 3, 4, 0, 1)) 
    deformable_ddf = deformable_ddf * [-1, -1, 1]
    deformable_image = nib.Nifti1Image(deformable_ddf, affine, header)        
        
    epoch = ith_models[i].split('/')[-1].split('epoch')[-1].split('.pth')[0]
    outname_deformable = "output/GIN/" + outfolder + "/DeformableWarp/" + mris[ith-1].split('/')[-1].split('.')[0] + '_' + str(epoch) + "_warp.nii.gz"
    nib.save(deformable_image, outname_deformable)
    
    moving_labels = torch.from_numpy(nib.load(labels[ith-1]).get_fdata()).type('torch.FloatTensor').unsqueeze(0).unsqueeze(0)
    pred_labels = warp_nearest(moving_labels.to(device), ddfs[1]).cpu().numpy()       
    outname_label = "output/GIN/" + outfolder + "/Labels/" +  mris[ith-1].split('/')[-1].split('.')[0] + "_" + epoch + "_labels.nii.gz"
    labels_image = nib.Nifti1Image(pred_labels.squeeze(), affine, header)
    nib.save(labels_image, outname_label)

['models/ith_null/null_ith_10_gin.pth', 'models/ith_null/null_ith_11_gin.pth', 'models/ith_null/null_ith_12_gin.pth', 'models/ith_null/null_ith_13_gin.pth', 'models/ith_null/null_ith_14_gin.pth', 'models/ith_null/null_ith_15_gin.pth', 'models/ith_null/null_ith_16_gin.pth', 'models/ith_null/null_ith_17_gin.pth', 'models/ith_null/null_ith_18_gin.pth', 'models/ith_null/null_ith_19_gin.pth', 'models/ith_null/null_ith_1_gin.pth', 'models/ith_null/null_ith_20_gin.pth', 'models/ith_null/null_ith_21_gin.pth', 'models/ith_null/null_ith_22_gin.pth', 'models/ith_null/null_ith_23_gin.pth', 'models/ith_null/null_ith_24_gin.pth', 'models/ith_null/null_ith_25_gin.pth', 'models/ith_null/null_ith_26_gin.pth', 'models/ith_null/null_ith_27_gin.pth', 'models/ith_null/null_ith_2_gin.pth', 'models/ith_null/null_ith_3_gin.pth', 'models/ith_null/null_ith_4_gin.pth', 'models/ith_null/null_ith_5_gin.pth', 'models/ith_null/null_ith_6_gin.pth', 'models/ith_null/null_ith_7_gin.pth', 'models/ith_null/null_ith_8_gin

In [ ]:
mris = sorted(glob(os.path.join("dataset3", "GIN", "MRI", "*_affine.nii.gz")))
for i in range(len(mris)):
    ith = i+1
    cmd = 'python3 reg_train.py -a -w 1 0 1 -lr 0.0001 -t localzero -e 250 -noaug -d ginaffine -o test_ith_' + str(ith) + '_gin'
    cmd = cmd.split(' ')
    cmd.extend(['-ith', str(ith)])
    subprocess.call(cmd)

In [9]:
ith_models = sorted(glob(os.path.join("models", "ith_zero_new", "test_ith_*.pth")))
labels = sorted(glob(os.path.join("dataset3", "GIN", "Labels", "*_affine_convert.nii.gz")))
mris = sorted(glob(os.path.join("dataset3", "GIN", "MRI", "*_affine.nii.gz")))
print(ith_models)
outfolder = 'pairwise_zero_nosym_fix'
warp_nearest = Warp("nearest", "zeros")
atlas_name = "dataset3/Atlas/P56_Atlas_128_norm_id.nii.gz"    
atlas_labels = "dataset3/Atlas/P56_Annotation_128_norm_id.nii.gz"
ants_warp = sorted(glob(os.path.join("output", "Feminad", "ANTS", "DeformableWarp", "*.nii.gz")))
affine = nib.load(ants_warp[0]).affine
header = nib.load(ants_warp[0]).header         
affine_atlas = nib.load(atlas_labels).affine
header_atlas = nib.load(atlas_labels).header 
device = getDevice()
for i, _ in enumerate(ith_models): 
    ith_model = ith_models[i].split('models/')[-1]
    ith = int(ith_model.split('test_ith_')[-1].split('_')[0])
    print(ith_model)
    print(ith)
    print(mris[ith-1])
    print('-'*10)
    outname = "output/GIN/" + outfolder + "/MRI_N4_Registration_Deformable/" + mris[ith-1].split('/')[-1].split('.')[0]
    pred_images, ddfs = reg_mri.main(ith_model, mris[ith-1], outname, False, True, "localzero", newmodel=False, sym=False)
    
    pred_image = pred_images[0]
    #predsym_image = pred_images[1]
    moving_image = ddfs[0].cpu()
    
    deformable_ddf = ddfs[1].cpu()
    deformable_ddf = deformable_ddf.cpu().numpy()
    deformable_ddf = np.transpose(deformable_ddf, (2, 3, 4, 0, 1)) 
    deformable_ddf = deformable_ddf * [-1, -1, 1]
    deformable_image = nib.Nifti1Image(deformable_ddf, affine, header)        
    
    #deformable_sym_ddf = ddfs[2].cpu()
    #deformable_sym_ddf = deformable_sym_ddf.cpu().numpy()
    #deformable_sym_ddf = np.transpose(deformable_sym_ddf, (2, 3, 4, 0, 1)) 
    #deformable_sym_ddf = deformable_sym_ddf * [-1, -1, 1]
    #deformable_sym_image = nib.Nifti1Image(deformable_sym_ddf, affine, header)
        
    outname_deformable = "output/GIN/" + outfolder + "/DeformableWarp/" + mris[ith-1].split('/')[-1].split('.')[0] + "_warp.nii.gz"
    nib.save(deformable_image, outname_deformable)

    #outname_sym_deformable = "output/GIN/" + outfolder + "/DeformableWarp/" + mris[ith-1].split('/')[-1].split('.')[0] + "_invwarp.nii.gz"
    #nib.save(deformable_sym_image, outname_sym_deformable)
    
    moving_labels = torch.from_numpy(nib.load(labels[ith-1]).get_fdata()).type('torch.FloatTensor').unsqueeze(0).unsqueeze(0)
    pred_labels = warp_nearest(moving_labels.to(device), ddfs[1]).cpu().numpy()       
    outname_label = "output/GIN/" + outfolder + "/Labels/" +  mris[ith-1].split('/')[-1].split('.')[0] + "_labels.nii.gz"
    labels_image = nib.Nifti1Image(pred_labels.squeeze(), affine, header)
    nib.save(labels_image, outname_label)
    
    #moving_labels = torch.from_numpy(nib.load(atlas_labels).get_fdata()).type('torch.FloatTensor').unsqueeze(0).unsqueeze(0)
    #pred_labels = warp_nearest(moving_labels.to(device), ddfs[2]).cpu().numpy()       
    #outname_label = "output/GIN/" + outfolder + "/LabelsAtlas/" +  mris[ith-1].split('/')[-1].split('.')[0] + "_labelsatlas.nii.gz"
    #labels_image = nib.Nifti1Image(pred_labels.squeeze(), affine_atlas, header_atlas)
    #nib.save(labels_image, outname_label)

['models/ith_zero_new/test_ith_10_gin.pth', 'models/ith_zero_new/test_ith_11_gin.pth', 'models/ith_zero_new/test_ith_12_gin.pth', 'models/ith_zero_new/test_ith_13_gin.pth', 'models/ith_zero_new/test_ith_14_gin.pth', 'models/ith_zero_new/test_ith_15_gin.pth', 'models/ith_zero_new/test_ith_16_gin.pth', 'models/ith_zero_new/test_ith_17_gin.pth', 'models/ith_zero_new/test_ith_18_gin.pth', 'models/ith_zero_new/test_ith_19_gin.pth', 'models/ith_zero_new/test_ith_1_gin.pth', 'models/ith_zero_new/test_ith_20_gin.pth', 'models/ith_zero_new/test_ith_21_gin.pth', 'models/ith_zero_new/test_ith_22_gin.pth', 'models/ith_zero_new/test_ith_23_gin.pth', 'models/ith_zero_new/test_ith_24_gin.pth', 'models/ith_zero_new/test_ith_25_gin.pth', 'models/ith_zero_new/test_ith_26_gin.pth', 'models/ith_zero_new/test_ith_27_gin.pth', 'models/ith_zero_new/test_ith_2_gin.pth', 'models/ith_zero_new/test_ith_3_gin.pth', 'models/ith_zero_new/test_ith_4_gin.pth', 'models/ith_zero_new/test_ith_5_gin.pth', 'models/ith_zer

In [10]:
mris = sorted(glob(os.path.join("dataset3", "GIN", "MRI", "*_affine.nii.gz")))
#labels = sorted(glob(os.path.join('dataset3', 'GIN', 'Labels', "*_affine_convert.nii.gz")))
#for i in range(len(mris)):
#    ith = i+1
#    cmd = 'python3 reg_train.py -a -w 1 0 1 -lr 0.0001 -t local -e 150 -d ginaffine -o ith_best_aug_nosym_' + str(ith) + '_gin'
#    cmd = cmd.split(' ')
#    cmd.extend(['-ith', str(ith)])
#    subprocess.call(cmd)
    
ith_models = sorted(glob(os.path.join("models", "ith_best_aug_nosym", "ith_best_aug_nosym_*.pth")))
print(ith_models)
outfolder = 'ith_best_aug_nosym'
warp_nearest = Warp("nearest", "zeros")
atlas_name = "dataset3/Atlas/P56_Atlas_128_norm_id.nii.gz"    
atlas_labels = "dataset3/Atlas/P56_Annotation_128_norm_id.nii.gz"
ants_warp = sorted(glob(os.path.join("output", "Feminad", "ANTS", "DeformableWarp", "*.nii.gz")))
affine = nib.load(ants_warp[0]).affine
header = nib.load(ants_warp[0]).header         
affine_atlas = nib.load(atlas_labels).affine
header_atlas = nib.load(atlas_labels).header 
device = getDevice()
for i, _ in enumerate(ith_models): 
    ith_model = ith_models[i].split('models/')[-1]
    ith = int(ith_model.split('ith_best_aug_nosym_')[-1].split('_')[0])
    print(ith_model)
    print(ith)
    print(mris[ith-1])
    print('-'*10)
    outname = "output/GIN/" + outfolder + "/MRI_N4_Registration_Deformable/" + mris[ith-1].split('/')[-1].split('.')[0]
    pred_images, ddfs = reg_mri.main(ith_model, mris[ith-1], outname, False, True, "local", newmodel=False, sym=False)
    
    pred_image = pred_images[0]
    #predsym_image = pred_images[1]
    moving_image = ddfs[0].cpu()
    
    deformable_ddf = ddfs[1].cpu()
    deformable_ddf = deformable_ddf.cpu().numpy()
    deformable_ddf = np.transpose(deformable_ddf, (2, 3, 4, 0, 1)) 
    deformable_ddf = deformable_ddf * [-1, -1, 1]
    deformable_image = nib.Nifti1Image(deformable_ddf, affine, header)        
    
    #deformable_sym_ddf = ddfs[2].cpu()
    #deformable_sym_ddf = deformable_sym_ddf.cpu().numpy()
    #deformable_sym_ddf = np.transpose(deformable_sym_ddf, (2, 3, 4, 0, 1)) 
    #deformable_sym_ddf = deformable_sym_ddf * [-1, -1, 1]
    #deformable_sym_image = nib.Nifti1Image(deformable_sym_ddf, affine, header)
        
    outname_deformable = "output/GIN/" + outfolder + "/DeformableWarp/" + mris[ith-1].split('/')[-1].split('.')[0] + "_warp.nii.gz"
    nib.save(deformable_image, outname_deformable)

    #outname_sym_deformable = "output/GIN/" + outfolder + "/DeformableWarp/" + mris[ith-1].split('/')[-1].split('.')[0] + "_invwarp.nii.gz"
    #nib.save(deformable_sym_image, outname_sym_deformable)
    
    moving_labels = torch.from_numpy(nib.load(labels[ith-1]).get_fdata()).type('torch.FloatTensor').unsqueeze(0).unsqueeze(0)
    pred_labels = warp_nearest(moving_labels.to(device), ddfs[1]).cpu().numpy()       
    outname_label = "output/GIN/" + outfolder + "/Labels/" +  mris[ith-1].split('/')[-1].split('.')[0] + "_labels.nii.gz"
    labels_image = nib.Nifti1Image(pred_labels.squeeze(), affine, header)
    nib.save(labels_image, outname_label)
    
    #moving_labels = torch.from_numpy(nib.load(atlas_labels).get_fdata()).type('torch.FloatTensor').unsqueeze(0).unsqueeze(0)
    #pred_labels = warp_nearest(moving_labels.to(device), ddfs[2]).cpu().numpy()       
    #outname_label = "output/GIN/" + outfolder + "/LabelsAtlas/" +  mris[ith-1].split('/')[-1].split('.')[0] + "_labelsatlas.nii.gz"
    #labels_image = nib.Nifti1Image(pred_labels.squeeze(), affine_atlas, header_atlas)
    #nib.save(labels_image, outname_label)

['models/ith_best_aug_nosym/ith_best_aug_nosym_10_gin_1.0-0.0-1.0.pth', 'models/ith_best_aug_nosym/ith_best_aug_nosym_11_gin_1.0-0.0-1.0.pth', 'models/ith_best_aug_nosym/ith_best_aug_nosym_12_gin_1.0-0.0-1.0.pth', 'models/ith_best_aug_nosym/ith_best_aug_nosym_13_gin_1.0-0.0-1.0.pth', 'models/ith_best_aug_nosym/ith_best_aug_nosym_14_gin_1.0-0.0-1.0.pth', 'models/ith_best_aug_nosym/ith_best_aug_nosym_15_gin_1.0-0.0-1.0.pth', 'models/ith_best_aug_nosym/ith_best_aug_nosym_16_gin_1.0-0.0-1.0.pth', 'models/ith_best_aug_nosym/ith_best_aug_nosym_17_gin_1.0-0.0-1.0.pth', 'models/ith_best_aug_nosym/ith_best_aug_nosym_18_gin_1.0-0.0-1.0.pth', 'models/ith_best_aug_nosym/ith_best_aug_nosym_19_gin_1.0-0.0-1.0.pth', 'models/ith_best_aug_nosym/ith_best_aug_nosym_1_gin_1.0-0.0-1.0.pth', 'models/ith_best_aug_nosym/ith_best_aug_nosym_20_gin_1.0-0.0-1.0.pth', 'models/ith_best_aug_nosym/ith_best_aug_nosym_21_gin_1.0-0.0-1.0.pth', 'models/ith_best_aug_nosym/ith_best_aug_nosym_22_gin_1.0-0.0-1.0.pth', 'model

In [ ]:
ith_models = sorted(glob(os.path.join("models", "ith_zero", "ith_zero_*.pth")))
print(ith_models)
outfolder = 'ith_zero'
warp_nearest = Warp("nearest", "zeros")
atlas_name = "dataset3/Atlas/P56_Atlas_128_norm_id.nii.gz"    
atlas_labels = "dataset3/Atlas/P56_Annotation_128_norm_id.nii.gz"
ants_warp = sorted(glob(os.path.join("output", "Feminad", "ANTS", "DeformableWarp", "*.nii.gz")))
affine = nib.load(ants_warp[0]).affine
header = nib.load(ants_warp[0]).header         
affine_atlas = nib.load(atlas_labels).affine
header_atlas = nib.load(atlas_labels).header 
device = getDevice()
for i, _ in enumerate(ith_models): 
    ith_model = ith_models[i].split('models/')[-1]
    ith = int(ith_model.split('ith_zero_')[-1].split('_')[0])
    print(ith_model)
    print(ith)
    print(mris[ith-1])
    print('-'*10)
    outname = "output/GIN/" + outfolder + "/MRI_N4_Registration_Deformable/" + mris[ith-1].split('/')[-1].split('.')[0]
    pred_images, ddfs = reg_mri.main(ith_model, mris[ith-1], outname, False, True, "local", newmodel=False, sym=True)
    
    pred_image = pred_images[0]
    #predsym_image = pred_images[1]
    moving_image = ddfs[0].cpu()
    
    deformable_ddf = ddfs[1].cpu()
    deformable_ddf = deformable_ddf.cpu().numpy()
    deformable_ddf = np.transpose(deformable_ddf, (2, 3, 4, 0, 1)) 
    deformable_ddf = deformable_ddf * [-1, -1, 1]
    deformable_image = nib.Nifti1Image(deformable_ddf, affine, header)        
    
    #deformable_sym_ddf = ddfs[2].cpu()
    #deformable_sym_ddf = deformable_sym_ddf.cpu().numpy()
    #deformable_sym_ddf = np.transpose(deformable_sym_ddf, (2, 3, 4, 0, 1)) 
    #deformable_sym_ddf = deformable_sym_ddf * [-1, -1, 1]
    #deformable_sym_image = nib.Nifti1Image(deformable_sym_ddf, affine, header)
        
    outname_deformable = "output/GIN/" + outfolder + "/DeformableWarp/" + mris[ith-1].split('/')[-1].split('.')[0] + "_warp.nii.gz"
    nib.save(deformable_image, outname_deformable)

    #outname_sym_deformable = "output/GIN/" + outfolder + "/DeformableWarp/" + mris[ith-1].split('/')[-1].split('.')[0] + "_invwarp.nii.gz"
    #nib.save(deformable_sym_image, outname_sym_deformable)
    
    moving_labels = torch.from_numpy(nib.load(labels[ith-1]).get_fdata()).type('torch.FloatTensor').unsqueeze(0).unsqueeze(0)
    pred_labels = warp_nearest(moving_labels.to(device), ddfs[1]).cpu().numpy()       
    outname_label = "output/GIN/" + outfolder + "/Labels/" +  mris[ith-1].split('/')[-1].split('.')[0] + "_labels.nii.gz"
    labels_image = nib.Nifti1Image(pred_labels.squeeze(), affine, header)
    nib.save(labels_image, outname_label)
    
    #moving_labels = torch.from_numpy(nib.load(atlas_labels).get_fdata()).type('torch.FloatTensor').unsqueeze(0).unsqueeze(0)
    #pred_labels = warp_nearest(moving_labels.to(device), ddfs[2]).cpu().numpy()       
    #outname_label = "output/GIN/" + outfolder + "/LabelsAtlas/" +  mris[ith-1].split('/')[-1].split('.')[0] + "_labelsatlas.nii.gz"
    #labels_image = nib.Nifti1Image(pred_labels.squeeze(), affine_atlas, header_atlas)
    #nib.save(labels_image, outname_label)

In [ ]:
#mris = sorted(glob(os.path.join("dataset3", "GIN", "MRI", "*_affine.nii.gz")))
#labels = sorted(glob(os.path.join('dataset3', 'GIN', 'Labels', "*_affine_convert.nii.gz")))
#for i in range(len(mris)):
#    ith = i+1
#    cmd = 'python3 reg_train.py -a -w 1 0 1 -lr 0.0001 -sym 0.001 -t local -e 150 -d ginaffine -o ith_best_aug_' + str(ith) + '_gin'
#    cmd = cmd.split(' ')
#    cmd.extend(['-ith', str(ith)])
#    subprocess.call(cmd)
    
ith_models = sorted(glob(os.path.join("models", "ith_best_aug", "ith_best_aug_*.pth")))
print(ith_models)
outfolder = 'ith_best_aug'
warp_nearest = Warp("nearest", "zeros")
atlas_name = "dataset3/Atlas/P56_Atlas_128_norm_id.nii.gz"    
atlas_labels = "dataset3/Atlas/P56_Annotation_128_norm_id.nii.gz"
ants_warp = sorted(glob(os.path.join("output", "Feminad", "ANTS", "DeformableWarp", "*.nii.gz")))
affine = nib.load(ants_warp[0]).affine
header = nib.load(ants_warp[0]).header         
affine_atlas = nib.load(atlas_labels).affine
header_atlas = nib.load(atlas_labels).header 
device = getDevice()
for i, _ in enumerate(ith_models): 
    ith_model = ith_models[i].split('models/')[-1]
    ith = int(ith_model.split('ith_best_aug_')[-1].split('_')[0])
    print(ith_model)
    print(ith)
    print(mris[ith-1])
    print('-'*10)
    outname = "output/GIN/" + outfolder + "/MRI_N4_Registration_Deformable/" + mris[ith-1].split('/')[-1].split('.')[0]
    pred_images, ddfs = reg_mri.main(ith_model, mris[ith-1], outname, False, True, "local", newmodel=False, sym=True)
    
    pred_image = pred_images[0]
    predsym_image = pred_images[1]
    moving_image = ddfs[0].cpu()
    
    deformable_ddf = ddfs[1].cpu()
    deformable_ddf = deformable_ddf.cpu().numpy()
    deformable_ddf = np.transpose(deformable_ddf, (2, 3, 4, 0, 1)) 
    deformable_ddf = deformable_ddf * [-1, -1, 1]
    deformable_image = nib.Nifti1Image(deformable_ddf, affine, header)        
    
    deformable_sym_ddf = ddfs[2].cpu()
    deformable_sym_ddf = deformable_sym_ddf.cpu().numpy()
    deformable_sym_ddf = np.transpose(deformable_sym_ddf, (2, 3, 4, 0, 1)) 
    deformable_sym_ddf = deformable_sym_ddf * [-1, -1, 1]
    deformable_sym_image = nib.Nifti1Image(deformable_sym_ddf, affine, header)

        
    outname_deformable = "output/GIN/" + outfolder + "/DeformableWarp/" + mris[ith-1].split('/')[-1].split('.')[0] + "_warp.nii.gz"
    nib.save(deformable_image, outname_deformable)

    outname_sym_deformable = "output/GIN/" + outfolder + "/DeformableWarp/" + mris[ith-1].split('/')[-1].split('.')[0] + "_invwarp.nii.gz"
    nib.save(deformable_sym_image, outname_sym_deformable)
    
    moving_labels = torch.from_numpy(nib.load(labels[ith-1]).get_fdata()).type('torch.FloatTensor').unsqueeze(0).unsqueeze(0)
    pred_labels = warp_nearest(moving_labels.to(device), ddfs[1]).cpu().numpy()       
    outname_label = "output/GIN/" + outfolder + "/Labels/" +  mris[ith-1].split('/')[-1].split('.')[0] + "_labels.nii.gz"
    labels_image = nib.Nifti1Image(pred_labels.squeeze(), affine, header)
    nib.save(labels_image, outname_label)
    
    moving_labels = torch.from_numpy(nib.load(atlas_labels).get_fdata()).type('torch.FloatTensor').unsqueeze(0).unsqueeze(0)
    pred_labels = warp_nearest(moving_labels.to(device), ddfs[2]).cpu().numpy()       
    outname_label = "output/GIN/" + outfolder + "/LabelsAtlas/" +  mris[ith-1].split('/')[-1].split('.')[0] + "_labelsatlas.nii.gz"
    labels_image = nib.Nifti1Image(pred_labels.squeeze(), affine_atlas, header_atlas)
    nib.save(labels_image, outname_label)

In [5]:
lambdas1 = [1.0]
for l1 in lambdas1:
    j = 3
    mris = sorted(glob(os.path.join("dataset3", "GIN", "MRI", "*_affine.nii.gz")))
    labels = sorted(glob(os.path.join('dataset3', 'GIN', 'Labels', "*_affine_convert.nii.gz")))
    #for i in range(len(mris)):
    #    ith = i+1
    #    cmd = 'python3 reg_train.py -a -w 1 0 ' + str(l1) + ' -lr 0.0001 -t local -e 150 -d ginaffine -o ith_l1_' + str(j) + '_' + str(ith) + '_gin'
    #    print(cmd)
    #    cmd = cmd.split(' ')
    #    cmd.extend(['-ith', str(ith)])
    #    subprocess.call(cmd)

    ith_models = sorted(glob(os.path.join("models", "ith_lambda1", "ith_l1_" + str(j) + "_*.pth")))
    print(ith_models)
    outfolder = 'ith_lambda1_' + str(j)
    print(outfolder)
    warp_nearest = Warp("nearest", "zeros")
    atlas_name = "dataset3/Atlas/P56_Atlas_128_norm_id.nii.gz"    
    atlas_labels = "dataset3/Atlas/P56_Annotation_128_norm_id.nii.gz"
    ants_warp = sorted(glob(os.path.join("output", "Feminad", "ANTS", "DeformableWarp", "*.nii.gz")))
    affine = nib.load(ants_warp[0]).affine
    header = nib.load(ants_warp[0]).header         
    affine_atlas = nib.load(atlas_labels).affine
    header_atlas = nib.load(atlas_labels).header 
    device = getDevice()
    for i, _ in enumerate(ith_models): 
        ith_model = ith_models[i].split('models/')[-1]
        ith = int(ith_model.split('ith_l1_' + str(j) + '_')[-1].split('_')[0])
        print(ith_model)
        print(ith)
        print(mris[ith-1])
        print('-'*10)
        outname = "output/GIN/" + outfolder + "/MRI_N4_Registration_Deformable/" + mris[ith-1].split('/')[-1].split('.')[0]
        pred_images, ddfs = reg_mri.main(ith_model, mris[ith-1], outname, False, True, "local", newmodel=False, sym=False)


        deformable_ddf = ddfs[1].cpu()
        deformable_ddf = deformable_ddf.cpu().numpy()
        deformable_ddf = np.transpose(deformable_ddf, (2, 3, 4, 0, 1)) 
        deformable_ddf = deformable_ddf * [-1, -1, 1]
        deformable_image = nib.Nifti1Image(deformable_ddf, affine, header)        

        outname_deformable = "output/GIN/" + outfolder + "/DeformableWarp/" + mris[ith-1].split('/')[-1].split('.')[0] + "_warp.nii.gz"
        nib.save(deformable_image, outname_deformable)

        moving_labels = torch.from_numpy(nib.load(labels[ith-1]).get_fdata()).type('torch.FloatTensor').unsqueeze(0).unsqueeze(0)
        pred_labels = warp_nearest(moving_labels.to(device), ddfs[1]).cpu().numpy()       
        outname_label = "output/GIN/" + outfolder + "/Labels/" +  mris[ith-1].split('/')[-1].split('.')[0] + "_labels.nii.gz"
        labels_image = nib.Nifti1Image(pred_labels.squeeze(), affine, header)
        nib.save(labels_image, outname_label)

['models/ith_lambda1/ith_l1_3_10_gin_1.0-0.0-1.0.pth', 'models/ith_lambda1/ith_l1_3_11_gin_1.0-0.0-1.0.pth', 'models/ith_lambda1/ith_l1_3_12_gin_1.0-0.0-1.0.pth', 'models/ith_lambda1/ith_l1_3_13_gin_1.0-0.0-1.0.pth', 'models/ith_lambda1/ith_l1_3_14_gin_1.0-0.0-1.0.pth', 'models/ith_lambda1/ith_l1_3_15_gin_1.0-0.0-1.0.pth', 'models/ith_lambda1/ith_l1_3_16_gin_1.0-0.0-1.0.pth', 'models/ith_lambda1/ith_l1_3_17_gin_1.0-0.0-1.0.pth', 'models/ith_lambda1/ith_l1_3_18_gin_1.0-0.0-1.0.pth', 'models/ith_lambda1/ith_l1_3_19_gin_1.0-0.0-1.0.pth', 'models/ith_lambda1/ith_l1_3_1_gin_1.0-0.0-1.0.pth', 'models/ith_lambda1/ith_l1_3_20_gin_1.0-0.0-1.0.pth', 'models/ith_lambda1/ith_l1_3_21_gin_1.0-0.0-1.0.pth', 'models/ith_lambda1/ith_l1_3_22_gin_1.0-0.0-1.0.pth', 'models/ith_lambda1/ith_l1_3_23_gin_1.0-0.0-1.0.pth', 'models/ith_lambda1/ith_l1_3_24_gin_1.0-0.0-1.0.pth', 'models/ith_lambda1/ith_l1_3_25_gin_1.0-0.0-1.0.pth', 'models/ith_lambda1/ith_l1_3_26_gin_1.0-0.0-1.0.pth', 'models/ith_lambda1/ith_l1_3

In [2]:
import subprocess
from glob import glob
import os
import monai
from monai.networks.blocks import Warp
import nibabel as nib
import reg_mri
import numpy as np
import torch
from utils import getDevice

In [7]:
lambdas2 = [1000]
for l2 in lambdas2:
    j = 7
    mris = sorted(glob(os.path.join("dataset3", "GIN", "MRI", "*_affine.nii.gz")))
    labels = sorted(glob(os.path.join('dataset3', 'GIN', 'Labels', "*_affine_convert.nii.gz")))
    for i in range(len(mris)):
        ith = i+1
        cmd = 'python3 reg_train.py -a -noaug -w 1 0 1 -sym ' + str(l2) + ' -lr 0.0001 -t local -e 150 -d ginaffine -o ith_lambda2/ith_l2_' + str(j) + '_' + str(ith) + '_gin'
        print(cmd)
        cmd = cmd.split(' ')
        cmd.extend(['-ith', str(ith)])
        print(cmd)
        subprocess.call(cmd)

    ith_models = sorted(glob(os.path.join("models", "ith_lambda2", "ith_l2_" + str(j) + "_*.pth")))
    print(ith_models)
    outfolder = 'ith_lambda2_' + str(j)
    print(outfolder)
    warp_nearest = Warp("nearest", "zeros")
    atlas_name = "dataset3/Atlas/P56_Atlas_128_norm_id.nii.gz"    
    atlas_labels = "dataset3/Atlas/P56_Annotation_128_norm_id.nii.gz"
    ants_warp = sorted(glob(os.path.join("output", "Feminad", "ANTS", "DeformableWarp", "*.nii.gz")))
    affine = nib.load(ants_warp[0]).affine
    header = nib.load(ants_warp[0]).header         
    affine_atlas = nib.load(atlas_labels).affine
    header_atlas = nib.load(atlas_labels).header 
    device = getDevice()
    for i, _ in enumerate(ith_models): 
        ith_model = ith_models[i].split('models/')[-1]
        ith = int(ith_model.split('ith_l2_' + str(j) + '_')[-1].split('_')[0])
        print(ith_model)
        print(ith)
        print(mris[ith-1])
        print('-'*10)
        outname = "output/GIN/" + outfolder + "/MRI_N4_Registration_Deformable/" + mris[ith-1].split('/')[-1].split('.')[0]
        pred_images, ddfs = reg_mri.main(ith_model, mris[ith-1], outname, False, True, "local", newmodel=False, sym=True)

        pred_image = pred_images[0]
        predsym_image = pred_images[1]
        moving_image = ddfs[0].cpu()

        deformable_ddf = ddfs[1].cpu()
        deformable_ddf = deformable_ddf.cpu().numpy()
        deformable_ddf = np.transpose(deformable_ddf, (2, 3, 4, 0, 1)) 
        deformable_ddf = deformable_ddf * [-1, -1, 1]
        deformable_image = nib.Nifti1Image(deformable_ddf, affine, header)        

        deformable_sym_ddf = ddfs[2].cpu()
        deformable_sym_ddf = deformable_sym_ddf.cpu().numpy()
        deformable_sym_ddf = np.transpose(deformable_sym_ddf, (2, 3, 4, 0, 1)) 
        deformable_sym_ddf = deformable_sym_ddf * [-1, -1, 1]
        deformable_sym_image = nib.Nifti1Image(deformable_sym_ddf, affine, header)

        outname_deformable = "output/GIN/" + outfolder + "/DeformableWarp/" + mris[ith-1].split('/')[-1].split('.')[0] + "_warp.nii.gz"
        nib.save(deformable_image, outname_deformable)

        outname_sym_deformable = "output/GIN/" + outfolder + "/DeformableWarp/" + mris[ith-1].split('/')[-1].split('.')[0] + "_invwarp.nii.gz"
        nib.save(deformable_sym_image, outname_sym_deformable)

        moving_labels = torch.from_numpy(nib.load(labels[ith-1]).get_fdata()).type('torch.FloatTensor').unsqueeze(0).unsqueeze(0)
        pred_labels = warp_nearest(moving_labels.to(device), ddfs[1]).cpu().numpy()       
        outname_label = "output/GIN/" + outfolder + "/Labels/" +  mris[ith-1].split('/')[-1].split('.')[0] + "_labels.nii.gz"
        labels_image = nib.Nifti1Image(pred_labels.squeeze(), affine, header)
        nib.save(labels_image, outname_label)

        moving_labels = torch.from_numpy(nib.load(atlas_labels).get_fdata()).type('torch.FloatTensor').unsqueeze(0).unsqueeze(0)
        pred_labels = warp_nearest(moving_labels.to(device), ddfs[2]).cpu().numpy()       
        outname_label = "output/GIN/" + outfolder + "/LabelsAtlas/" +  mris[ith-1].split('/')[-1].split('.')[0] + "_labelsatlas.nii.gz"
        labels_image = nib.Nifti1Image(pred_labels.squeeze(), affine_atlas, header_atlas)
        nib.save(labels_image, outname_label)

python3 reg_train.py -a -noaug -w 1 0 1 -sym 1000 -lr 0.0001 -t local -e 150 -d ginaffine -o ith_lambda2/ith_l2_7_1_gin
['python3', 'reg_train.py', '-a', '-noaug', '-w', '1', '0', '1', '-sym', '1000', '-lr', '0.0001', '-t', 'local', '-e', '150', '-d', 'ginaffine', '-o', 'ith_lambda2/ith_l2_7_1_gin', '-ith', '1']
=> Training local registration model
=> Performing to-atlas registration.
=> Using old model (d4channels16)
=> Use DVF2DDF
=> Using symmetric models
=> Using i-th image for 1-image training
=> Saving to ith_lambda2/ith_l2_7_1_gin.pth
=> Weights are: [1.0, 0.0, 1.0]
=> Using GIN affine registered dataset.
----------
epoch 0/150


/home/valentini/dev/Mousenet/venv/lib/python3.8/site-packages/monai/networks/blocks/warp.py:66: UserWarning:

monai.networks.blocks.Warp: Using PyTorch native grid_sample.

Loading dataset: 100%|██████████| 1/1 [00:00<00:00, 9754.20it/s]


valid: loss: 0.1768 - metric: 0.0000 -- img: 0.0845, ddf: 0.0000 || sym_img: 0.0923 - sym_ddf: 0.0000 - sym_inv: 0.0000 || dice: 0.9453
best loss 0.1768 at epoch 0
----------
epoch 1/150
train: loss: 0.1768 - metric: 0.0000 -- img: 0.0845, ddf: 0.0000 || sym_img: 0.0923 - sym_ddf: 0.0000 - sym_inv: 0.0000 || dice: 0.9453
valid: loss: 1.4389 - metric: 0.0000 -- img: 0.0834, ddf: 0.0000 || sym_img: 0.0912 - sym_ddf: 0.0000 - sym_inv: 1.2643 || dice: 0.9453
----------
epoch 2/150
train: loss: 30.1723 - metric: 0.0000 -- img: 0.0803, ddf: 0.0000 || sym_img: 0.0883 - sym_ddf: 0.0000 - sym_inv: 30.0037 || dice: 0.9453
valid: loss: 0.2041 - metric: 0.0000 -- img: 0.0844, ddf: 0.0000 || sym_img: 0.0922 - sym_ddf: 0.0000 - sym_inv: 0.0275 || dice: 0.9453
----------
epoch 3/150
train: loss: 1.8516 - metric: 0.0000 -- img: 0.0835, ddf: 0.0000 || sym_img: 0.0913 - sym_ddf: 0.0000 - sym_inv: 1.6768 || dice: 0.9453
valid: loss: 0.3267 - metric: 0.0000 -- img: 0.0844, ddf: 0.0000 || sym_img: 0.0921 -

valid: loss: 0.1953 - metric: 0.0000 -- img: 0.0844, ddf: 0.0000 || sym_img: 0.0923 - sym_ddf: 0.0000 - sym_inv: 0.0186 || dice: 0.9453
----------
epoch 29/150
train: loss: 0.3042 - metric: 0.0000 -- img: 0.0844, ddf: 0.0000 || sym_img: 0.0922 - sym_ddf: 0.0000 - sym_inv: 0.1276 || dice: 0.9453
valid: loss: 0.2245 - metric: 0.0000 -- img: 0.0844, ddf: 0.0000 || sym_img: 0.0922 - sym_ddf: 0.0000 - sym_inv: 0.0478 || dice: 0.9453
----------
epoch 30/150
train: loss: 0.3609 - metric: 0.0000 -- img: 0.0843, ddf: 0.0000 || sym_img: 0.0922 - sym_ddf: 0.0000 - sym_inv: 0.1845 || dice: 0.9453
valid: loss: 0.2549 - metric: 0.0000 -- img: 0.0844, ddf: 0.0000 || sym_img: 0.0922 - sym_ddf: 0.0000 - sym_inv: 0.0783 || dice: 0.9453
----------
epoch 31/150
train: loss: 0.4022 - metric: 0.0000 -- img: 0.0843, ddf: 0.0000 || sym_img: 0.0921 - sym_ddf: 0.0000 - sym_inv: 0.2258 || dice: 0.9453
valid: loss: 0.2770 - metric: 0.0000 -- img: 0.0844, ddf: 0.0000 || sym_img: 0.0922 - sym_ddf: 0.0000 - sym_inv:

train: loss: 0.1823 - metric: 0.0000 -- img: 0.0845, ddf: 0.0000 || sym_img: 0.0923 - sym_ddf: 0.0000 - sym_inv: 0.0056 || dice: 0.9453
valid: loss: 0.1798 - metric: 0.0000 -- img: 0.0845, ddf: 0.0000 || sym_img: 0.0923 - sym_ddf: 0.0000 - sym_inv: 0.0030 || dice: 0.9453
----------
epoch 57/150
train: loss: 0.1809 - metric: 0.0000 -- img: 0.0845, ddf: 0.0000 || sym_img: 0.0923 - sym_ddf: 0.0000 - sym_inv: 0.0041 || dice: 0.9453
valid: loss: 0.1794 - metric: 0.0000 -- img: 0.0845, ddf: 0.0000 || sym_img: 0.0923 - sym_ddf: 0.0000 - sym_inv: 0.0027 || dice: 0.9453
----------
epoch 58/150
train: loss: 0.1799 - metric: 0.0000 -- img: 0.0845, ddf: 0.0000 || sym_img: 0.0923 - sym_ddf: 0.0000 - sym_inv: 0.0032 || dice: 0.9453
valid: loss: 0.1794 - metric: 0.0000 -- img: 0.0845, ddf: 0.0000 || sym_img: 0.0923 - sym_ddf: 0.0000 - sym_inv: 0.0026 || dice: 0.9453
----------
epoch 59/150
train: loss: 0.1794 - metric: 0.0000 -- img: 0.0845, ddf: 0.0000 || sym_img: 0.0923 - sym_ddf: 0.0000 - sym_inv:

...

=> Training local registration model
=> Performing to-atlas registration.
=> Using old model (d4channels16)
=> Use DVF2DDF
=> Using symmetric models
=> Using i-th image for 1-image training
=> Saving to ith_lambda2/ith_l2_7_4_gin.pth
=> Weights are: [1.0, 0.0, 1.0]
=> Using GIN affine registered dataset.
----------
epoch 0/150
valid: loss: 0.1858 - metric: 0.0000 -- img: 0.0919, ddf: 0.0000 || sym_img: 0.0940 - sym_ddf: 0.0000 - sym_inv: 0.0000 || dice: 0.9430
best loss 0.1858 at epoch 0
----------
epoch 1/150
train: loss: 0.1858 - metric: 0.0000 -- img: 0.0919, ddf: 0.0000 || sym_img: 0.0940 - sym_ddf: 0.0000 - sym_inv: 0.0000 || dice: 0.9430
valid: loss: 1.1387 - metric: 0.0000 -- img: 0.0909, ddf: 0.0000 || sym_img: 0.0931 - sym_ddf: 0.0000 - sym_inv: 0.9547 || dice: 0.9430
----------
epoch 2/150
train: loss: 30.4853 - metric: 0.0000 -- img: 0.0876, ddf: 0.0000 || sym_img: 0.0902 - sym_ddf: 0.0000 - sym_inv: 30.3075 || dice: 0.9430
valid: loss: 0.2113 - metric: 0.0000 -- img: 0.0918

train: loss: 0.2402 - metric: 0.0000 -- img: 0.0918, ddf: 0.0000 || sym_img: 0.0939 - sym_ddf: 0.0000 - sym_inv: 0.0545 || dice: 0.9430
valid: loss: 0.1954 - metric: 0.0000 -- img: 0.0918, ddf: 0.0000 || sym_img: 0.0939 - sym_ddf: 0.0000 - sym_inv: 0.0097 || dice: 0.9430
----------
epoch 28/150
train: loss: 0.2848 - metric: 0.0000 -- img: 0.0918, ddf: 0.0000 || sym_img: 0.0938 - sym_ddf: 0.0000 - sym_inv: 0.0992 || dice: 0.9430
valid: loss: 0.2310 - metric: 0.0000 -- img: 0.0918, ddf: 0.0000 || sym_img: 0.0939 - sym_ddf: 0.0000 - sym_inv: 0.0454 || dice: 0.9430
----------
epoch 29/150
train: loss: 0.3650 - metric: 0.0000 -- img: 0.0917, ddf: 0.0000 || sym_img: 0.0938 - sym_ddf: 0.0000 - sym_inv: 0.1795 || dice: 0.9430
valid: loss: 0.2769 - metric: 0.0000 -- img: 0.0917, ddf: 0.0000 || sym_img: 0.0938 - sym_ddf: 0.0000 - sym_inv: 0.0913 || dice: 0.9430
----------
epoch 30/150
train: loss: 0.4386 - metric: 0.0000 -- img: 0.0917, ddf: 0.0000 || sym_img: 0.0937 - sym_ddf: 0.0000 - sym_inv:

train: loss: 0.1919 - metric: 0.0000 -- img: 0.0918, ddf: 0.0000 || sym_img: 0.0939 - sym_ddf: 0.0000 - sym_inv: 0.0061 || dice: 0.9430
valid: loss: 0.1904 - metric: 0.0000 -- img: 0.0918, ddf: 0.0000 || sym_img: 0.0939 - sym_ddf: 0.0000 - sym_inv: 0.0046 || dice: 0.9430
----------
epoch 56/150
train: loss: 0.1910 - metric: 0.0000 -- img: 0.0918, ddf: 0.0000 || sym_img: 0.0939 - sym_ddf: 0.0000 - sym_inv: 0.0052 || dice: 0.9430
valid: loss: 0.1901 - metric: 0.0000 -- img: 0.0918, ddf: 0.0000 || sym_img: 0.0939 - sym_ddf: 0.0000 - sym_inv: 0.0044 || dice: 0.9430
----------
epoch 57/150
train: loss: 0.1906 - metric: 0.0000 -- img: 0.0918, ddf: 0.0000 || sym_img: 0.0939 - sym_ddf: 0.0000 - sym_inv: 0.0048 || dice: 0.9430
valid: loss: 0.1902 - metric: 0.0000 -- img: 0.0918, ddf: 0.0000 || sym_img: 0.0939 - sym_ddf: 0.0000 - sym_inv: 0.0045 || dice: 0.9430
----------
epoch 58/150
train: loss: 0.1907 - metric: 0.0000 -- img: 0.0918, ddf: 0.0000 || sym_img: 0.0939 - sym_ddf: 0.0000 - sym_inv:

Traceback (most recent call last):
  File "reg_train.py", line 481, in <module>
    train(args.output, args.dataset, args.finetuning, args.continuetraining, args.batchsize, args.epochs,
  File "reg_train.py", line 119, in train
    ddf, pred_image, pred_label, dvf, ddf2, dvf2 = model(data)
  File "/home/valentini/dev/Mousenet/venv/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1110, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/valentini/dev/Mousenet/reg_model.py", line 123, in forward
    ddf = self.dvf2ddf(dvf)
  File "/home/valentini/dev/Mousenet/venv/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1110, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/valentini/dev/Mousenet/venv/lib/python3.8/site-packages/monai/networks/blocks/warp.py", line 155, in forward
    ddf = ddf + self.warp_layer(image=ddf, ddf=ddf)
  File "/home/valentini/dev/Mousenet/venv/lib/python3.8/site-packages/torch/nn/modules/module.py", li

KeyboardInterrupt: 